In [6]:
import os
import logging
from pathlib import Path
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv

### Meeting Summarization

In [7]:
import os
import logging
from pathlib import Path
from typing import Optional
from openai import OpenAI


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY', ''))

def build_prompt(meeting_transcript: str) -> str:
    try:
        template = {
            'system_role': "You are an AI assistant that summarizes the entire meeting by category.",
            'output_format': [
                "Next meeting date and time: OOO",
                "Topic: OOO",
                "Task List: OOO",
                "Team members' opinions: OOO",
                "Mentor feedback: OOO"
            ]
        }

        prompt = f""" {template['system_role']}
Please summarize the meeting content below using the following format and please write them down in English:

[Output Example]
{chr(10).join(template['output_format'])}

[Full Meeting Transcript]
{meeting_transcript}

[Summary Result] """
        return prompt

    except Exception as e:
        logging.error(f"Failed to build prompt: {str(e)}")
        raise

def summarize_meeting(transcript: str):
    prompt = build_prompt(transcript)


    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an AI assistant that summarizes the entire meeting by category."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )


    return response.choices[0].message.content.strip()


def get_user_transcript():
    print("Please enter the meeting transcript (press Enter twice to finish):")
    lines = []
    while True:
        line = input()
        if line.strip() == "" and (not lines or lines[-1].strip() == ""):
            break
        lines.append(line)
    return "\n".join(lines).strip()


if __name__ == "__main__":

    if not os.environ.get('OPENAI_API_KEY'):
        api_key = input("Enter your OpenAI API key: ")
        os.environ['OPENAI_API_KEY'] = api_key
        client = OpenAI(api_key=api_key)


    try:
        meeting_transcript = get_user_transcript()
        summary = summarize_meeting(meeting_transcript)
        print(summary)
    except Exception as e:
        logging.error(f"An error occurred while summarizing the meeting: {str(e)}")

Please enter the meeting transcript (press Enter twice to finish):


2025-05-12 17:07:16,617 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Next meeting date and time: Next Tuesday at 10 AM  
Topic: API Risk Insurance Model Design  
Task List:  
1. Finalize data collection and establish guidelines for API classification.  
2. Refine the insurance premium pricing model based on traffic segments.  
3. Develop risk scenarios based on recent fintech issues.  

Team members' opinions:  
- Data collection is mostly complete, but there are issues with SLA violations and missing data in smaller APIs.  
- The classification of APIs into 'major' and 'minor' based on call volume and failure rates is a good approach, though more statistical validation is needed.  
- The insurance premium model needs adjustments, particularly for real-time APIs where prediction errors are larger.  
- Risk scenarios are being developed, but there is a need to incorporate more recent fintech-related issues.  

Mentor feedback:  
- Previous scenarios were deemed too general; the team is encouraged to include specific recent incidents, such as authenticati

### Task Alignment

In [9]:
def build_task_prompt(task_list, team_traits):
    tasks_str = "\n".join(f"{i+1}. {task.strip()}" for i, task in enumerate(task_list.splitlines()))
    return f"""
You are an AI assistant tasked with assigning responsibilities based on each team member’s traits.

[Team Member Traits]
{team_traits}

[Task List]
{tasks_str}

- Give the entire task assignment result in english.
- Do not use gendered pronouns such as "he" or "she." Use the person's name instead in the justification.
- Assign exactly one team member to each task, along with a brief justification.
- Your justification must reflect the individual’s actual traits.
- Do not assign tasks based on traits the person does not have (e.g., do not cite creativity for someone who is not described as creative).
- Use proper language to describe trait levels (e.g., “moderate,” “high,” etc.).
- If the number of tasks is fewer than the number of team members, create additional tasks as needed. 
  If there are more tasks than members, merge simple or similar tasks so that each task is assigned to one person.


[Output Example]
1. Task - Assigned to: Name (Justification: aligned with specific traits)
"""

def assign_tasks(task_list, team_traits):
    prompt = build_task_prompt(task_list, team_traits)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return response.choices[0].message.content.strip()

print("\nEnter team member traits. Example:")
print("00: Intuitive type, creative, moderate activity level, performance-driven with charismatic leadership, cautious in relationships, moderate execution\n...")

team_traits_input = []
while True:
    line = input("> ")
    if not line.strip():
        break
    team_traits_input.append(line)
team_traits = "\n".join(team_traits_input)

print("\nEnter the task list (one task per line, press Enter twice to finish):")
task_list_input = []
while True:
    line = input("> ")
    if not line.strip():
        break
    task_list_input.append(line)
task_list = "\n".join(task_list_input)

try:
    result = assign_tasks(task_list, team_traits)
    print("\nTask assignment result:\n")
    print(result)
except Exception as e:
    logging.error(f"Task assignment failed: {str(e)}")



Enter team member traits. Example:
00: Intuitive type, creative, moderate activity level, performance-driven with charismatic leadership, cautious in relationships, moderate execution
...

Enter the task list (one task per line, press Enter twice to finish):


2025-05-12 17:08:24,412 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Task assignment result:

1. Finalize data collection and establish guidelines for API classification - Assigned to: 윤조 (Justification: 윤조 has strong analytical skills and moderate execution ability, making them well-suited for organizing data and creating structured guidelines.)

2. Refine the insurance premium pricing model based on traffic segments - Assigned to: 혜주 (Justification: 혜주's creativity and performance-oriented charismatic leadership will help in innovating and improving the pricing model effectively.)

3. Develop risk scenarios based on recent fintech issues - Assigned to: 정아 (Justification: 정아's high sense of responsibility and strong execution ability, combined with a realistic approach, make them ideal for developing thorough and practical risk scenarios.)

4. Additional Task: Conduct a team review of the finalized API guidelines and pricing model - Assigned to: 현서 (Justification: 현서's careful relationship-oriented approach and moderate execution ability will facilita